Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape(
        (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [13]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [16]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
            [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
            [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [17]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.376161
Minibatch accuracy: 6.2%
Validation accuracy: 9.0%
Minibatch loss at step 50: 2.034088
Minibatch accuracy: 31.2%
Validation accuracy: 43.1%
Minibatch loss at step 100: 0.985778
Minibatch accuracy: 68.8%
Validation accuracy: 63.2%
Minibatch loss at step 150: 1.225320
Minibatch accuracy: 75.0%
Validation accuracy: 73.9%
Minibatch loss at step 200: 0.856515
Minibatch accuracy: 68.8%
Validation accuracy: 77.4%
Minibatch loss at step 250: 0.846469
Minibatch accuracy: 62.5%
Validation accuracy: 76.9%
Minibatch loss at step 300: 0.535842
Minibatch accuracy: 81.2%
Validation accuracy: 79.0%
Minibatch loss at step 350: 0.911263
Minibatch accuracy: 68.8%
Validation accuracy: 79.0%
Minibatch loss at step 400: 0.967623
Minibatch accuracy: 75.0%
Validation accuracy: 78.8%
Minibatch loss at step 450: 0.491029
Minibatch accuracy: 81.2%
Validation accuracy: 81.7%
Minibatch loss at step 500: 0.627737
Minibatch accuracy: 81.2%
Validation accuracy: 80.4%
Min

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [18]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
            [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
            [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        hidden = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                                padding='SAME')
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        hidden = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                                padding='SAME')
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [19]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.962266
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.225925
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 100: 1.833439
Minibatch accuracy: 25.0%
Validation accuracy: 42.0%
Minibatch loss at step 150: 1.628520
Minibatch accuracy: 56.2%
Validation accuracy: 60.2%
Minibatch loss at step 200: 0.789304
Minibatch accuracy: 81.2%
Validation accuracy: 75.0%
Minibatch loss at step 250: 0.832132
Minibatch accuracy: 81.2%
Validation accuracy: 77.2%
Minibatch loss at step 300: 0.466263
Minibatch accuracy: 87.5%
Validation accuracy: 79.5%
Minibatch loss at step 350: 0.850434
Minibatch accuracy: 75.0%
Validation accuracy: 78.8%
Minibatch loss at step 400: 1.233523
Minibatch accuracy: 62.5%
Validation accuracy: 78.4%
Minibatch loss at step 450: 0.517089
Minibatch accuracy: 81.2%
Validation accuracy: 82.4%
Minibatch loss at step 500: 0.601813
Minibatch accuracy: 87.5%
Validation accuracy: 82.5%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

Solution
----
Note: The codes are refectored to make them more manageable similar to the one given [here](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/convolutional_network.py)

In [15]:
# Parameters
image_size=28
num_channels = 1
batch_size = 16
patch_size = 5
learning_rate = 0.05
num_steps = 1001
display_step = 50
#depth = 16
#num_hidden = 64

# Network Parameters
#n_inputs = image_size**2 # 784
n_classes = 10 # Number of output classes (A-J)
dropout = 1.0  # Dropout for regularization

# Tensorflow Graph Input
x = tf.placeholder(tf.float32, shape=[None, image_size, image_size, num_channels])
#x = tf.placeholder(tf.float32, [None, n_inputs])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [16]:
#Weights and Biases
# weights = {
#     # 5x5 conv, 1 input, 32 outputs
#     'wc1': tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, 32])),
#     # 5x5 conv, 32 inputs, 64 outputs
#     'wc2': tf.Variable(tf.truncated_normal([patch_size, patch_size, 32, 64])),
#     # fully connected, 7*7*64 inputs, 1024 outputs
#     'wf1': tf.Variable(tf.truncated_normal([(image_size//4)*(image_size//4)*64, 1024])),
#     # 1024 inputs, 10 outputs (class prediction)
#     'out': tf.Variable(tf.truncated_normal([1024, n_classes]))
# }
# biases = {
#     'bc1': tf.Variable(tf.truncated_normal([32])),
#     'bc2': tf.Variable(tf.truncated_normal([64])),
#     'bf1': tf.Variable(tf.truncated_normal([1024])),
#     'out': tf.Variable(tf.truncated_normal([n_classes]))
# }

weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, 16])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.truncated_normal([patch_size, patch_size, 16, 16])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wf1': tf.Variable(tf.truncated_normal([(image_size//4)*(image_size//4)*16, 64])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.truncated_normal([64, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.truncated_normal([16])),
    'bc2': tf.Variable(tf.truncated_normal([16])),
    'bf1': tf.Variable(tf.truncated_normal([64])),
    'out': tf.Variable(tf.truncated_normal([n_classes]))
}

Create functions for simplicity

In [17]:
def conv2d(x, W, b, strides=1):
    """Computes a convolutional layer with relu"""
    conv = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    conv = tf.nn.relu(conv + b)
    return conv

def maxpool2d(x, k=2):
    """Applies a max pooling of size k (maxPool2D) to a given data."""
    max_pool = tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                              padding='SAME')
    return max_pool

The function below creates a conv_net model.

In [18]:
def conv_net_model(data, weights, biases, dropout):
    """Creates a convolutional networks model and returns its outputs"""
    # Reshape the input image if not already done.
    #data = tf.reshape(data, shape=[-1, image_size, image_size, num_channels])

    ## 1. Convolutional Layer
    conv1 = conv2d(data, weights['wc1'], biases['bc1'])
    # Max Pooling (For down-sampling)
    conv1 = maxpool2d(conv1)
    
    ## 2. Convolutional Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (For down-sampling)
    conv2 = maxpool2d(conv2)
    
    ## 3. Fully Connected Layer
    # Reshape conv2 to fit fully connected layer input
    fc1 = tf.reshape(conv2, shape=[-1, weights['wf1'].get_shape().as_list()[0]])
    # Multiply with the given weights and add the biases
    fc1 = tf.add(tf.matmul(fc1, weights['wf1']), biases['bf1'])
    # Apply relu
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout for regularization (Dropouts are applied during training only)
    fc1 = tf.nn.dropout(fc1, dropout)
    
    # Output Layer, For Class Prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    
    return out

Let's now construct our model

In [19]:
# Construct the model
predictions = conv_net_model(x, weights, biases, keep_prob)

# Define the loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(predictions, y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

# Evaluate the model
correct_pred = tf.equal(tf.arg_max(predictions, 1), tf.arg_max(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))*100.0

# Initialize the variables
init = tf.initialize_all_variables()

Now, we will launch the graph as follows.

In [21]:
with tf.Session() as session:
    session.run(init)
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_x = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_y = train_labels[offset:(offset + batch_size), :]
        # Run optimization op (backprop)
        feed_dict = {x : batch_x, y : batch_y, keep_prob : dropout}
        session.run(optimizer, feed_dict=feed_dict)
#         if (step % display_step == 0):
#             # Calculate batch loss and accuracy (here, dropout = 1.0)
#             feed_dict = {x : batch_x, y : batch_y, keep_prob : 1.0}
#             loss, acc = session.run([loss, accuracy], feed_dict=feed_dict)
#             print('Minibatch loss at step %d: %f' % (step, loss))
#             print('Minibatch accuracy: %.1f%%' % acc)
#             #Accuracy for Validation Set
#             acc = session.run(accuracy, feed_dict={x: valid_dataset, y: valid_labels,
#                                                   keep_prob: 1.0})
#             print('Validation accuracy: %.1f%%' % acc)

    print("Optimization Finished!")
    #Accuracy for Test Set
    acc = session.run(accuracy, feed_dict={x: test_dataset, y: test_labels,
                                          keep_prob: 1.0})
    print('Test accuracy: %.1f%%' % acc)

Optimization Finished!
Test accuracy: 10.0%


In [22]:
from __future__ import print_function

import tensorflow as tf

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [31]:
mnist.train.images.shape[0]/batch_size, mnist.train.labels.shape
type(mnist.train)


tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet

In [26]:
# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)


# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')



In [27]:
# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [28]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [29]:
# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.}))

Iter 1280, Minibatch Loss= 31373.187500, Training Accuracy= 0.35156
Iter 2560, Minibatch Loss= 12991.867188, Training Accuracy= 0.46875
Iter 3840, Minibatch Loss= 11733.712891, Training Accuracy= 0.53906
Iter 5120, Minibatch Loss= 5963.323242, Training Accuracy= 0.69531
Iter 6400, Minibatch Loss= 4516.456055, Training Accuracy= 0.77344
Iter 7680, Minibatch Loss= 7360.002930, Training Accuracy= 0.71875
Iter 8960, Minibatch Loss= 3374.771973, Training Accuracy= 0.79688
Iter 10240, Minibatch Loss= 2715.493652, Training Accuracy= 0.84375
Iter 11520, Minibatch Loss= 1496.087891, Training Accuracy= 0.88281
Iter 12800, Minibatch Loss= 3973.423584, Training Accuracy= 0.79688
Iter 14080, Minibatch Loss= 2097.506592, Training Accuracy= 0.88281
Iter 15360, Minibatch Loss= 2225.569092, Training Accuracy= 0.89844
Iter 16640, Minibatch Loss= 3045.694824, Training Accuracy= 0.84375
Iter 17920, Minibatch Loss= 848.955994, Training Accuracy= 0.92188
Iter 19200, Minibatch Loss= 1184.372192, Training Acc

[Deep MNIST For Experts](https://www.tensorflow.org/versions/r0.11/tutorials/mnist/pros/)

What we will accomplish in this tutorial:

* Create a softmax regression function that is a model for recognizing MNIST digits, based on looking at every pixel in the image
* Use Tensorflow to train the model to recognize digits by having it "look" at thousands of examples (and run our first Tensorflow session to do so)
* Check the model's accuracy with our test data
* Build, train, and test a multilayer convolutional neural network to improve the results

In [1]:
import tensorflow as tf

#### Load MNIST Data

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


#### Start TensorFlow InteractiveSession

In [3]:
sess = tf.InteractiveSession()

#### A. Build a Softmax Regression Model

* Placeholders

In [4]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)

* Variables

In [5]:
W = tf.Variable(tf.truncated_normal([784,10], stddev=0.1))
b = tf.Variable(tf.truncated_normal([10], stddev=0.1))

sess.run(tf.initialize_all_variables())

* Predicted Class and Loss Function

In [6]:
y_ = tf.matmul(x,W) + b
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_, y))

* Train the Model

In [10]:
optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

# Training the model is accomplished by repeatedly running the optimizer
for i in range(1000):
    batch = mnist.train.next_batch(100)
    optimizer.run(feed_dict={x: batch[0], y: batch[1]})

* Evaluate the Model

In [11]:
correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Compute the accuracy of the model on the test data
print(accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels}))

0.9189


#### B. Build a Multilayer Convolutional Network

* Weight and Bias Initialization

In [5]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

* Convolution and Pooling

In [6]:
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

* Construct Model with 2 Convolutional Layers and 1 fully connected layer 

In [7]:
def model(x, weights, biases, dropout):
    
    # Reshape the input data
    x = tf.reshape(x, [-1,28,28,1])
    
    # 1st convolutional layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # 2nd Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wf1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wf1']), biases['bf1'])
    fc1 = tf.nn.relu(fc1)
    
    # Apply Dropout (To reduce overfitting)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Readout Layer (for class prediction)
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out   

In [8]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': weight_variable([5, 5, 1, 32]),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': weight_variable([5, 5, 32, 64]),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wf1': weight_variable([7*7*64, 1024]),
    # 1024 inputs, 10 outputs (class prediction)
    'out': weight_variable([1024, 10])
}

biases = {
    'bc1': bias_variable([32]),
    'bc2': bias_variable([64]),
    'bf1': bias_variable([1024]),
    'out': bias_variable([10])
}

* Train and Evaluate the Model

In [9]:
y_ = model(x, weights, biases, keep_prob)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_, y))
optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)

correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess.run(tf.initialize_all_variables())

In [11]:
for i in range(20000):
    batch = mnist.train.next_batch(50)

    sess.run(optimizer, feed_dict={x: batch[0], y: batch[1], keep_prob: 0.5})

    if i%100 == 0:
        loss, train_acc = sess.run([loss, accuracy], feed_dict={x: batch[0], y: batch[1], keep_prob: 1.0})
        #train_accuracy = accuracy.eval(feed_dict={
        #        x:batch[0], y: batch[1], keep_prob: 1.0})
        print("step %d, Loss %.4f, training accuracy %.2f"%(i, loss, train_acc))

print("Test accuracy %g"%accuracy.eval(feed_dict={
            x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0}))

TypeError: Fetch argument 4.6305132 has invalid type <type 'numpy.float32'>, must be a string or Tensor. (Can not convert a float32 into a Tensor or Operation.)